 ## Extraction of Table Formats, such as Bank Statements, into Structured Formats

 #### Overview:
 - Develop a system to extract structured information from documents like bank statements that contain table based formats. The extracted information should include personal details (e.g., name, bank) and financial transactions, organised into a key value structure for personal details and table formats for transactions. The output should be saved in formats like XLSX or CSV for further analysis and processing.
  
 #### Objective:
 - The system should extract personal details (e.g., name, bank name) from the top of the bank statement in a key value format and the transaction data in table format. The extracted data should then be saved in table formats like CSV or XLSX for ease of use in financial analysis or reporting systems.

 #### Data:
 - Various Bank statements or similar financial documents containing tabular data and personal details.

 #### Requirements:
 ##### Input:
 - Information Scanned or image based bank statements containing personal details at the top and tabular transaction data below.
  
 ##### Output:
 - Personal details in key value format (e.g., "Name: John Doe", "Bank: XYZ").
 - Transaction details extracted in a tabular format and saved as XLSX, CSV, or JSON.

In [1]:
# Install PaddleOCR package
! pip install paddlepaddle paddleocr

Import the Libraries

In [2]:
import os
import re
import json
import pandas as pd
from paddleocr import PaddleOCR

File Path of Input and Output Directories

In [3]:
image_path = r'Input_Dir\Extraction_Table_Formats\sample_table_format.png'  # Change this to your image path
output_path = r'Output_Dir\Extraction_table_format'   # Change this to your desired output directory
# Create the output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

In [4]:
# Initialize PaddleOCR model
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Perform OCR on the image
ocr_result = ocr.ocr(image_path, cls=True)

# Extracted text result
extracted_text = []
for line in ocr_result[0]:
    extracted_text.append(line[1][0])

# Combine the extracted lines into a single text block
text = '\n'.join(extracted_text)
print(text)

[2024/09/25 17:58:24] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\augus/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\augus/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None,

In [5]:
personal_details = {}

# Use regex to extract fields like Account Name, Account Number, Branch, etc.
patterns = {
    'Account Name': r"Account Name\s*(.*)",
    'Account Number': r"Account Number\s*:\s*(\d+)",
    'Account Description': r"Account Description\s*:\s*(.*)",
    'Branch': r"Branch\s*(.*)",
    'CIF No.': r"CIF No.\s*(\d+)",
    'IFS Code': r"IFS Code\s*(\w+)",
    'MICR Code': r"MICR Code\s*(\d+)",
    'Interest Rate': r"Interest Rate\(% p.a.\)\s*(\d+\.\d+)",
    'Drawing Power': r"Drawing Power\s*([0-9,.]+)",
    'MOD Balance': r"MOD Balance\s*([0-9,.]+)"
}

for key, pattern in patterns.items():
    match = re.search(pattern, text)
    if match:
        personal_details[key] = match.group(1).strip()

print(personal_details)

{'Account Name': 'Mrs.BADDU', 'Account Number': '00000061068722344', 'Account Description': 'REGULAR SB CHQ-INDIVIDUALS URAL-INR', 'Branch': 'MAKRANA', 'CIF No.': '71104584150', 'IFS Code': 'SBIN0031113', 'MICR Code': '341002005', 'Interest Rate': '3.25', 'Drawing Power': '0.00', 'MOD Balance': '0.00'}


In [6]:
# Split the data into lines
lines = text.strip().split('\n')

transactions = []
current_transaction = {}
previous_date = None

for line in lines:
    line = line.strip()
    
    # Check if the line is a valid date
    date_match = re.match(r'\d{1,2}\s?[A-Za-z]{3}\s?\d{4}|\d{1,2}[A-Za-z]{3}\d{4}', line)
    
    if date_match:
        current_date = date_match.group(0)

        # If there's a previous date, finalize the current transaction
        if previous_date and current_transaction:
            current_transaction['Transaction Date'] = previous_date
            current_transaction['Value Date'] = previous_date  # Ensure both dates are the same
            transactions.append(current_transaction)  # Save the current transaction
            current_transaction = {}  # Reset for the next transaction

        # Always update the current date
        previous_date = current_date  # Update the previous date
    
    elif re.match(r'^\d{1,3}(?:,\d{3})*(?:\.\d{2})?$', line):
        # Only process lines that match the float format
        amount = float(line.replace(',', ''))  # Remove commas and convert to float
        if 'Transaction Amount' not in current_transaction:  # Create Amount column
            current_transaction['Transaction Amount'] = amount
        else:
            # Set the balance only if the line is a valid numeric format
            balance = float(line.replace(',', ''))  # Convert balance to float
            if balance > amount:  # Ensure balance is always higher than amount
                current_transaction['Balance'] = balance
            else:
                # If the balance is not higher, retain the previously assigned balance
                current_transaction['Balance'] = current_transaction.get('Balance', balance)

# Finalizing the last transaction
if current_transaction:
    current_transaction['Transaction Date'] = previous_date
    current_transaction['Value Date'] = previous_date  # Ensure both dates are the same
    transactions.append(current_transaction)

# Convert transactions to DataFrame for easier manipulation
df_transactions = pd.DataFrame(transactions)

# Select and reorder columns
df_transactions = df_transactions[['Transaction Date', 'Value Date', 'Transaction Amount', 'Balance']]

# Format Amount and Balance to two decimal places
df_transactions['Transaction Amount'] = df_transactions['Transaction Amount'].map('{:.2f}'.format)
df_transactions['Balance'] = df_transactions['Balance'].map('{:.2f}'.format)

# Remove rows with '0.00', NaN, 'nan', and 'Nan'
df_transactions = df_transactions[~df_transactions['Transaction Amount'].isin(['0.00', 'nan', 'NaN'])]
df_transactions = df_transactions[~df_transactions['Balance'].isin(['0.00', 'nan', 'NaN'])]
df_transactions = df_transactions.dropna(subset=['Transaction Amount', 'Balance'])  # Drop rows with NaN values
print(df_transactions)

   Transaction Date  Value Date Transaction Amount   Balance
1        1 Sep 2019  1 Sep 2019            5000.00   7994.76
2        1 Sep 2019  1 Sep 2019            2000.00   9994.76
3        2 Sep 2019  2 Sep 2019           23850.00  33844.76
5        6 Sep 2019  6 Sep 2019            2000.00  29844.76
6        6 Sep 2019  6 Sep 2019            2000.00  27844.76
7        6 Sep 2019  6 Sep 2019            2000.00  25844.76
8          6Sep2019    6Sep2019            2000.00  23844.76
9        6 Sep 2019  6 Sep 2019             110.00  23734.76
10       9 Sep 2019  9 Sep 2019             340.00  24074.76


Save the Files in JSON, xlsx and CSV.

In [7]:
# Save personal details as JSON
personal_details_file_path = os.path.join(output_path, 'personal_details.json')
with open(personal_details_file_path, 'w') as json_file:
    json.dump(personal_details, json_file, indent=4)

# Save personal details as CSV
personal_details_df = pd.DataFrame.from_dict(personal_details, orient='index', columns=['Value'])
personal_details_df.index.name = 'Details'  # Rename the index to "Details"
personal_details_df.reset_index(inplace=True)  # Reset the index to make it a column
personal_details_csv_path = os.path.join(output_path, 'personal_details.csv')
personal_details_df.to_csv(personal_details_csv_path, index=False)

# Save personal details as XLSX
personal_details_xlsx_path = os.path.join(output_path, 'personal_details.xlsx')
personal_details_df.to_excel(personal_details_xlsx_path)

# Save transactions as JSON
transactions_path_json = os.path.join(output_path, 'transactions.json')
with open(transactions_path_json, 'w') as json_file:
    # Filter out transactions where both Transaction Amount and Balance are greater than 0
    valid_transactions = df_transactions[
        (df_transactions['Transaction Amount'].astype(float) > 0) & 
        (df_transactions['Balance'].astype(float) > 0)
    ]
    json.dump(valid_transactions.to_dict(orient='records'), json_file, indent=4)


# Save transactions as CSV
transactions_csv_path = os.path.join(output_path, 'transactions.csv')
df_transactions.to_csv(transactions_csv_path, index=False)

# Save transactions as XLSX
transactions_xlsx_path = os.path.join(output_path, 'transactions.xlsx')
df_transactions.to_excel(transactions_xlsx_path, index=False)

print("Files saved successfully!")

Files saved successfully!
